In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql import functions as F
from framework.feature_factory.helpers import Helpers
from channelDemoMarket import Store

spark = SparkSession.builder.appName('Test').getOrCreate()
spark.conf.set("spark.sql.shuffle.partitions", 96*2)

In [2]:
# Istantiate store
#store = Store(_snapshot_date = "2018-01-01") #This works, but no neted features.
store = Store(_snapshot_date = "2018-01-01")


In [3]:
# Get The feature factory
ff = store.ff

In [4]:
# Grab some sales features
mult_features, base_features = store.Sales().get_all()

In [5]:
### For fisualization only.
# Build a base dataframe from cores/sources
example_df = store.get_core("issuer")
print("n_rows: %d" %example_df.count())
print("n_cols: %d" %len(example_df.columns))
example_df.show(truncate=True, n=5)

n_rows: 5613
n_cols: 7
+--------------------+-----------+------------------+-----------+--------------------+----------------+----------------+
|header_capture_month|issuer_name|chip_indicator_uid|product_uid|         description|purchase_txn_amt|purchase_txn_cnt|
+--------------------+-----------+------------------+-----------+--------------------+----------------+----------------+
| 2016-09-01 00:00:00| Royal Bank|                 7|          1|     Hardware Stores|   7.127944081E7|          919933|
| 2017-04-01 00:00:00| Scotiabank|                 9|          1|Drug Stores, Phar...|   1.672346653E7|          784273|
| 2018-12-01 00:00:00| Royal Bank|                 3|          1|Grocery Stores, S...|       100088.58|            1194|
| 2018-08-01 00:00:00|       CIBC|                 7|          1|     Hardware Stores|   6.163271108E7|          706047|
| 2017-09-01 00:00:00|       CIBC|                 3|          1|Fuel Dispenser, A...|             0.0|               0|
+--------

In [6]:
### For visualization only.
# Build a base dataframe from cores/sources
bank_df = store.get_core("bank_id").alias('bank')
print("n_rows: %d" %bank_df.count())
print("n_cols: %d" %len(bank_df.columns))
bank_df.show(truncate=True, n=5)

n_rows: 5
n_cols: 2
+--------------------+---------+
|         issuer_name|issuer_id|
+--------------------+---------+
|          Royal Bank|        1|
|BMO Bank of Montreal|        2|
|          Scotiabank|        3|
|     TD Canada Trust|        4|
|                CIBC|        5|
+--------------------+---------+



In [8]:
# Build a base dataframe from cores/sources
store_sales_df = store.get_core("issuer").filter(col('purchase_txn_amt') > 0).alias('clean_amount')

In [9]:
print("n_rows: %d" %store_sales_df.count())
print("n_cols: %d" %len(store_sales_df.columns))
store_sales_df.show(truncate=True, n=5)

n_rows: 5342
n_cols: 7
+--------------------+---------------+------------------+-----------+--------------------+----------------+----------------+
|header_capture_month|    issuer_name|chip_indicator_uid|product_uid|         description|purchase_txn_amt|purchase_txn_cnt|
+--------------------+---------------+------------------+-----------+--------------------+----------------+----------------+
| 2016-09-01 00:00:00|     Royal Bank|                 7|          1|     Hardware Stores|   7.127944081E7|          919933|
| 2017-04-01 00:00:00|     Scotiabank|                 9|          1|Drug Stores, Phar...|   1.672346653E7|          784273|
| 2018-12-01 00:00:00|     Royal Bank|                 3|          1|Grocery Stores, S...|       100088.58|            1194|
| 2018-08-01 00:00:00|           CIBC|                 7|          1|     Hardware Stores|   6.163271108E7|          706047|
| 2018-10-01 00:00:00|TD Canada Trust|                 3|          1|Eating Places, Re...|           9

### Continue with example

Make a join!

In [11]:
base_df = store_sales_df.join(bank_df, ['issuer_name'])\
  .select('clean_amount.*', 'bank.issuer_id')
base_df.show(truncate=True, n=5)

+---------------+--------------------+------------------+-----------+--------------------+----------------+----------------+---------+
|    issuer_name|header_capture_month|chip_indicator_uid|product_uid|         description|purchase_txn_amt|purchase_txn_cnt|issuer_id|
+---------------+--------------------+------------------+-----------+--------------------+----------------+----------------+---------+
|     Royal Bank| 2016-09-01 00:00:00|                 7|          1|     Hardware Stores|   7.127944081E7|          919933|        1|
|     Scotiabank| 2017-04-01 00:00:00|                 9|          1|Drug Stores, Phar...|   1.672346653E7|          784273|        3|
|     Royal Bank| 2018-12-01 00:00:00|                 3|          1|Grocery Stores, S...|       100088.58|            1194|        1|
|           CIBC| 2018-08-01 00:00:00|                 7|          1|     Hardware Stores|   6.163271108E7|          706047|        5|
|TD Canada Trust| 2018-10-01 00:00:00|                 

### Show distinct categoricals

In [12]:
base_df.select("chip_indicator_uid").distinct().collect()

[Row(chip_indicator_uid=5),
 Row(chip_indicator_uid=7),
 Row(chip_indicator_uid=3),
 Row(chip_indicator_uid=9),
 Row(chip_indicator_uid=4),
 Row(chip_indicator_uid=2),
 Row(chip_indicator_uid=6)]

In [13]:
base_df.select("product_uid").distinct().collect()

[Row(product_uid=2), Row(product_uid=1)]

### Build a Features Dataframe

Here, we are simply calling the aggregation methods in mult_features to be applied over a group by on 'issuer_name'

In [14]:
feature_df = ff.append_features(base_df, groupBy_cols = ['issuer_name'], feature_sets=[mult_features])
#feature_df = ff.append_features(store_sales_df, groupBy_cols = ['issuer_name'], feature_sets=[mult_features])
feature_df.show()

+--------------------+--------------------+
|         issuer_name|           net_sales|
+--------------------+--------------------+
|BMO Bank of Montreal|  8.85825478646001E9|
|                CIBC|1.428790330563999E10|
|          Scotiabank|1.126012862541997...|
|          Royal Bank|1.919939227528001...|
|     TD Canada Trust|2.102817429950998...|
+--------------------+--------------------+



### Using Multipliers

We will create a new features dataframe using composite aggregations.

In [15]:
store.config.get_config('time_helpers').configs

{'snapshot_type': 'DAILY',
 'snapshot_date': '2018-01-01',
 'partition_col': None,
 'date_col': 'header_capture_month',
 'date_col_format': '%Y-%m-%d',
 'partition_col_format': '%Y%m',
 'date_filters': {'ranges': {'1m': {'start': '2017-12-01',
    'end': '2018-01-01'},
   '3m': {'start': '2017-10-01', 'end': '2018-01-01'}}},
 'partition_lower': '201710',
 'partition_upper': '201801'}

In [16]:
time_multipliers = store.get_daterange_multiplier()

In [17]:
mult_by_time_features = mult_features.multiply(time_multipliers, "STORE")

### Nested features
Now let's assume we have several cateogical columns for which we want to calculate aggregates.

The categorical multiplier allows you to either specific to which columns you wish to apply the multiplier as well as a minimum distinct values count n and an ignore list of columns and it will efficiently find all the columns with < n distinct values

In [24]:
categorical_multiplier = Helpers().get_categoricals_multiplier(df = store.get_core("issuer"),
                                                               col_list = ['product_uid'])

by_time_by_cat = mult_by_time_features.multiply(categorical_multiplier, "STORE")

In [29]:
feature_df = ff.append_features(base_df,
                                groupBy_cols=['issuer_name'],
                                feature_sets=[mult_features, mult_by_time_features, by_time_by_cat])

In [30]:
print("n cols = %d" %len(feature_df.columns))
feature_df.show(truncate=True, n=5)

n cols = 8
+--------------------+--------------------+--------------------+--------------------+--------------------------------+--------------------------------+--------------------------------+--------------------------------+
|         issuer_name|           net_sales|  STORE_1M_NET_SALES|  STORE_3M_NET_SALES|STORE_PRODUCT_UID-2_1M_NET_SALES|STORE_PRODUCT_UID-1_1M_NET_SALES|STORE_PRODUCT_UID-2_3M_NET_SALES|STORE_PRODUCT_UID-1_3M_NET_SALES|
+--------------------+--------------------+--------------------+--------------------+--------------------------------+--------------------------------+--------------------------------+--------------------------------+
|BMO Bank of Montreal|  8.85825478646001E9|2.8818181919000006E8| 7.857798393399997E8|                   5.359639188E7|            2.3458542731000003E8|                   5.369875249E7|             7.320810868499999E8|
|                CIBC|1.428790330563999E10|4.6480695909999996E8|1.2644895344200006E9|                             0.0

### Further nesting

In [31]:
categorical_multiplier_2 = Helpers().get_categoricals_multiplier(df = store.get_core("issuer"),
                                                               col_list = ['chip_indicator_uid'])

by_time_by_cat_by_cat = by_time_by_cat.multiply(categorical_multiplier_2, "STORE")

In [32]:
feature_df = ff.append_features(base_df,
                                groupBy_cols=['issuer_name'],
                                feature_sets=[by_time_by_cat_by_cat])

In [33]:
print("n cols = %d" %len(feature_df.columns))
feature_df.show(truncate=True, n=5)

n cols = 29
+--------------------+-----------------------------------------------------+-----------------------------------------------------+-----------------------------------------------------+-----------------------------------------------------+-----------------------------------------------------+-----------------------------------------------------+-----------------------------------------------------+-----------------------------------------------------+-----------------------------------------------------+-----------------------------------------------------+-----------------------------------------------------+-----------------------------------------------------+-----------------------------------------------------+-----------------------------------------------------+-----------------------------------------------------+-----------------------------------------------------+-----------------------------------------------------+------------------------------------------------